<a href="https://colab.research.google.com/github/cij1012/pytrorch/blob/master/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80_mydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sklearn에서 제공하는 iris 데이터 

In [0]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()

In [2]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [16]:
iris_df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
iris_df["target"] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [22]:
mask = iris_df['target'].isin(['2'])
iris_df = iris_df[~mask]
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1


In [23]:
data = iris_df
data = data.apply(
    lambda x : (x - x.mean()) / x.std()
)
data['target'] = iris_df['target']
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,-0.578153,0.837617,-1.007900,-1.036887,0
1,-0.889826,-0.206793,-1.007900,-1.036887,0
2,-1.201499,0.210971,-1.076887,-1.036887,0
3,-1.357336,0.002089,-0.938913,-1.036887,0
4,-0.733990,1.046500,-1.007900,-1.036887,0
...,...,...,...,...,...
95,0.356865,-0.206793,0.923736,0.732545,1
96,0.356865,-0.415675,0.923736,0.909488,1
97,1.136048,-0.415675,0.992723,0.909488,1
98,-0.578153,-1.251204,0.095892,0.555602,1


In [27]:
X,y = data.values[:,:-1],data.values[:,-1:]
print(X.shape, y.shape)

(100, 4) (100, 1)


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [0]:
class MyDataset(Dataset):
  def __init__(self):
    self.x_data = torch.tensor(X, dtype=torch.float)
    self.y_data = torch.tensor(y, dtype=torch.float)
  def __len__(self):
    return len(self.x_data)
  def __getitem__(self,idx):
    x = self.x_data[idx]
    y = self.y_data[idx]
    return x,y

In [0]:
dataset = MyDataset()
dataloader = DataLoader(dataset, batch_size=len(dataset))

In [0]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [0]:
model = BinaryClassifier()
optimizer = optim.SGD(model.parameters(), lr = 1)

In [34]:
nb_epochs = 2000
for epoch in range(nb_epochs+1):
  for x_train, y_train in dataloader:
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train) 

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
      prediction = hypothesis >= torch.FloatTensor([0.5])
      correct_prediction = prediction.float() == y_train
      accuracy = correct_prediction.sum().item() / len(correct_prediction)
      print('Epoch {:4d}/{} Cost :{:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy*100))

Epoch    0/2000 Cost :0.477921 Accuracy 99.00%
Epoch  100/2000 Cost :0.005876 Accuracy 100.00%
Epoch  200/2000 Cost :0.003199 Accuracy 100.00%
Epoch  300/2000 Cost :0.002241 Accuracy 100.00%
Epoch  400/2000 Cost :0.001740 Accuracy 100.00%
Epoch  500/2000 Cost :0.001430 Accuracy 100.00%
Epoch  600/2000 Cost :0.001218 Accuracy 100.00%
Epoch  700/2000 Cost :0.001063 Accuracy 100.00%
Epoch  800/2000 Cost :0.000945 Accuracy 100.00%
Epoch  900/2000 Cost :0.000852 Accuracy 100.00%
Epoch 1000/2000 Cost :0.000776 Accuracy 100.00%
Epoch 1100/2000 Cost :0.000714 Accuracy 100.00%
Epoch 1200/2000 Cost :0.000661 Accuracy 100.00%
Epoch 1300/2000 Cost :0.000616 Accuracy 100.00%
Epoch 1400/2000 Cost :0.000576 Accuracy 100.00%
Epoch 1500/2000 Cost :0.000542 Accuracy 100.00%
Epoch 1600/2000 Cost :0.000512 Accuracy 100.00%
Epoch 1700/2000 Cost :0.000485 Accuracy 100.00%
Epoch 1800/2000 Cost :0.000461 Accuracy 100.00%
Epoch 1900/2000 Cost :0.000440 Accuracy 100.00%
Epoch 2000/2000 Cost :0.000420 Accuracy 1